 # Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

### Step1: Get the table from Wikipedia page https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, and save it into a dataframe. Use BeautifulSoup package. 

In [1]:
### Install packages
!pip install msgpack
!pip install beautifulsoup4
!pip install lxml
from bs4 import BeautifulSoup
import requests
import pandas as pd


    100% |████████████████████████████████| 317kB 21.7MB/s 


In [2]:
### Scrape the page, get the table source code, use a "for" loop to extract the row and column information and finally convert them into a pandas dataframe.
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')

table = soup.find('table')
table_rows = table.find_all('tr')

l = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]  
    if row:
        l.append(row)
        
new_table = pd.DataFrame(l, columns=["Postcode", "Borough", "Neighborhood"])
new_table


,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


In [3]:
### dataframe new_table was generated
### check the shape of this new dataframe
new_table.shape
### So new_table has 289 rows (including the header row) and 3 columns, which matches the original table information on the wiki webpage.

(289, 3)

### Step2: Clean up and preprocess data in the table according to the requests in this assignment.

In [4]:
### Step2A: Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
new_table_2A = new_table[new_table.Borough != 'Not assigned']

In [5]:
new_table_2A.shape

(212, 3)

In [6]:
### Step2B: More than one neighborhood can exist in one postal code area. Combine different neighborhoods into one row and separate them with a comma. 
new_table_2B =pd.DataFrame(new_table_2A[['Postcode', 'Borough', 'Neighborhood']].groupby(['Postcode', 'Borough'])['Neighborhood'].apply(lambda x: "%s" % ', '.join(x)))
new_table_2B.reset_index(inplace = True)

In [7]:
new_table_2B.head()
### Now new_table_2B has different neighbourhoods under the same postcode combined in one row.

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [8]:
new_table_2B.shape

(103, 3)

In [9]:
### Step2C: If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
new_table_2B.loc[new_table_2B['Neighborhood'] == 'Not assigned', 'Neighborhood'] = new_table_2B.loc[new_table_2B['Neighborhood'] == 'Not assigned', 'Borough']
### Check and see if this change has been completed:
new_table_2B.loc[85]
### The result shows that now the Neighbourhood value with Borough = "Queen's Park" is also "Queen's Park".

Postcode                 M7A
Borough         Queen's Park
Neighborhood    Queen's Park
Name: 85, dtype: object

In [10]:
### Check the shape of the final dataframe:
new_table_final = new_table_2B
new_table_final.shape

(103, 3)

### Step3: Get geographical coordinates of each postal code from the csv file, and update the previous dataframe "new_table_final" by adding coordinates information from the csv file.

In [11]:
### Read in the csv file with location information
csv_url='http://cocl.us/Geospatial_data'
loc_data = pd.read_csv(csv_url)
loc_data.head()
#loc_data.shape

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
### Merge the location dataframe with the previously generated final table dataframe by matching the postal code
new_table_updated = new_table_final.merge(loc_data, left_on='Postcode', right_on='Postal Code', how='left').drop(['Postal Code'], axis = 1)

In [13]:
### Now a new dataframe with location information is generated
new_table_updated.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Step4: Explore and cluster the neighborhoods in Toronto

In [14]:
### Make a subset of data with only boroughs that contain the word Toronto 
### We will focus only on those boroughs
table_final = new_table_updated[new_table_updated['Borough'].str.contains("Toronto")] 
table_final.reset_index(drop = True, inplace = True)
table_final.shape

(38, 5)

In [15]:
### Take a look at the table
table_final

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049


In [16]:
### Import folium library
!conda install -c conda-forge folium=0.5.0 --yes 
import folium 


Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.0.2p             |       h470a237_1         3.1 MB  conda-forge
    certifi-2018.10.15         |        py36_1000         138 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.3.0               |             py_0          24 KB  conda-forge
    ca-certificates-2018.10.15 |       ha4d7672_0         135 KB  conda-forge
    conda-4.5.11               |        py36_1000         651 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-2.2.2               |        py36_1001         494 KB  conda-forge
    ------------------------------------------------------------
                         

In [17]:
### Define Foursquare Credentials and Version
CLIENT_ID = 'OXTXOI1FAI511MYK5AEVM3XQ2ZOHPI41JAFS2D0HYFLISMXP' # your Foursquare ID
CLIENT_SECRET = 'FKCB4YIV2KVAO4LQ2122P0JA5GNXTXB03NLBNBBNPMRORZR3' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OXTXOI1FAI511MYK5AEVM3XQ2ZOHPI41JAFS2D0HYFLISMXP
CLIENT_SECRET:FKCB4YIV2KVAO4LQ2122P0JA5GNXTXB03NLBNBBNPMRORZR3


In [18]:
### Explore Neighborhoods in Toronto
##  Create a function to get the top 100 nearby venues within a radius of 500 meters for all neighbourhoods in Toronto.
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [19]:
Toronto_venues = getNearbyVenues(names=table_final['Neighborhood'],
                                   latitudes=table_final['Latitude'],
                                   longitudes=table_final['Longitude'], radius=500)


The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

In [20]:
## Let's check the size of the resulting dataframe
print(Toronto_venues.shape)
Toronto_venues.head()

(1706, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Starbucks,43.678798,-79.298045,Coffee Shop
1,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
2,The Beaches,43.676357,-79.293031,St-Denis Studios Inc.,43.675031,-79.288022,Music Venue
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [21]:
## Let's check how many venues were returned for each neighborhood
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,57,57,57,57,57,57
"Brockton, Exhibition Place, Parkdale Village",21,21,21,21,21,21
Business reply mail Processing Centre969 Eastern,17,17,17,17,17,17
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",14,14,14,14,14,14
"Cabbagetown, St. James Town",49,49,49,49,49,49
Central Bay Street,87,87,87,87,87,87
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,16,16,16,16,16,16


In [22]:
## Let's find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 232 uniques categories.


In [23]:
### Analyze Each Neighborhood
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Thai Restaurant,Theater,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
Toronto_onehot.shape

(1706, 232)

In [25]:
# Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Thai Restaurant,Theater,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.040000,0.010000,0.000000,0.00000,0.000000,0.010000,0.000000,0.000000,0.010000,0.01
1,Berczy Park,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.017544,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
2,"Brockton, Exhibition Place, Parkdale Village",0.047619,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
3,Business reply mail Processing Centre969 Eastern,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.00,0.000000,0.000000,0.071429,0.071429,0.071429,0.142857,0.142857,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
5,"Cabbagetown, St. James Town",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.020408,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
6,Central Bay Street,0.011494,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.022989,0.000000,0.000000,0.00000,0.000000,0.011494,0.000000,0.000000,0.011494,0.00
7,"Chinatown, Grange Park, Kensington Market",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.010000,0.00000,0.000000,0.050000,0.000000,0.050000,0.010000,0.00
8,Christie,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
9,Church and Wellesley,0.011494,0.00,0.011494,0.011494,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.011494,0.000000,0.00000,0.000000,0.011494,0.011494,0.011494,0.000000,0.00


In [26]:
# write a function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [27]:
# let's create the new dataframe and display the top 10 venues for each neighborhood
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Café,Coffee Shop,American Restaurant,Thai Restaurant,Steakhouse,Gym,Bar,Restaurant,Hotel,Japanese Restaurant
1,Berczy Park,Coffee Shop,Cocktail Bar,Pub,Cheese Shop,Farmers Market,Bakery,Steakhouse,Seafood Restaurant,Café,Restaurant
2,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Breakfast Spot,Café,Performing Arts Venue,Gym,Stadium,Bar,Burrito Place,Caribbean Restaurant,Climbing Gym
3,Business reply mail Processing Centre969 Eastern,Light Rail Station,Burrito Place,Pizza Place,Recording Studio,Fast Food Restaurant,Farmers Market,Spa,Restaurant,Brewery,Auto Workshop
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Sculpture Garden,Harbor / Marina,Airport,Airport Food Court,Airport Gate,Boutique
5,"Cabbagetown, St. James Town",Restaurant,Coffee Shop,Chinese Restaurant,Pizza Place,Pub,Bakery,Italian Restaurant,Café,Market,Indian Restaurant
6,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Bubble Tea Shop,Japanese Restaurant,Burger Joint,Bar,Sandwich Place,Falafel Restaurant,Ice Cream Shop
7,"Chinatown, Grange Park, Kensington Market",Café,Bar,Chinese Restaurant,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Mexican Restaurant,Bakery,Coffee Shop,Dumpling Restaurant,Gaming Cafe
8,Christie,Café,Grocery Store,Park,Athletics & Sports,Convenience Store,Coffee Shop,Italian Restaurant,Baby Store,Nightclub,Restaurant
9,Church and Wellesley,Japanese Restaurant,Coffee Shop,Burger Joint,Sushi Restaurant,Gay Bar,Restaurant,Gastropub,Fast Food Restaurant,Nightclub,Men's Store


In [28]:
### Cluster Neighborhoods : Run k-means to cluster the neighborhood into 5 clusters
# set number of clusters
from sklearn.cluster import KMeans 
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [29]:
new_table_updated.head()
Toronto_data = table_final.drop(['Postcode'], axis = 1)
Toronto_data.head()



,Borough,Neighborhood,Latitude,Longitude
0,East Toronto,The Beaches,43.676357,-79.293031
1,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,East Toronto,Studio District,43.659526,-79.340923
4,Central Toronto,Lawrence Park,43.728020,-79.388790


In [30]:
# Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood
Toronto_merged = Toronto_data

# add clustering labels
Toronto_merged['Cluster Labels'] = kmeans.labels_

# merge Toronto_grouped with Toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.head() 

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,The Beaches,43.676357,-79.293031,0,Coffee Shop,Pub,Music Venue,Women's Store,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
1,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Dessert Shop,Bookstore,Brewery,Bubble Tea Shop,Pub,Café
2,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,0,Gym,Fish & Chips Shop,Ice Cream Shop,Steakhouse,Pub,Liquor Store,Sushi Restaurant,Italian Restaurant,Fast Food Restaurant,Burger Joint
3,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Italian Restaurant,Gastropub,American Restaurant,Bakery,Yoga Studio,Brewery,Bar,Juice Bar
4,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Swim School,Park,Dim Sum Restaurant,Bus Line,Women's Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


In [32]:
# Finally, let's visualize the resulting clusters

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors


In [41]:
# Let's view the map of Toronto
# Toronto latitude: 43.6532, 79.3832
latitude = 43.6532
longitude = -79.3832

# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

map_Toronto

### Final Step: generate the map to visualize the neighborhoods and how they cluster together

In [42]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters